In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import svm
import cv2
import csv
import os
from os import listdir
from os.path import isfile, join
import glob
from sklearn import cluster
from scipy.misc import *
import skimage.measure as sm
# import progressbar
import multiprocessing
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print('----Check output----\n', check_output(["ls", "./train/"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
dir_names = [d for d in listdir("./train/") if not isfile(join("./train/", d))]
print('\n ----Subdirectories----\n', dir_names)

#file_paths contains the file names for each category
#key has format: (Category,CatNumber,Path)
file_paths = {}
class_num = 0
cat_int = {}
int_cat = {}
prediction_output_list=[]
test_names=[f for f in listdir("./test_stg1/") if isfile(join("./test_stg1", f))]
for d in dir_names:
    fnames = [f for f in listdir("./train/"+d+"/") if isfile(join("./train/"+d+"/", f))]
    file_paths[(d, class_num, "./train/"+d+"/")] = fnames
    cat_int[d]=class_num
    int_cat[class_num]=d
    class_num += 1
for k in file_paths.keys():
    print(k)
    


----Check output----
 ALB
BET
DOL
LAG
NoF
OTHER
SHARK
YFT


 ----Subdirectories----
 ['NoF', 'YFT', 'LAG', 'OTHER', 'SHARK', 'DOL', 'ALB', 'BET']
('BET', 7, './train/BET/')
('ALB', 6, './train/ALB/')
('OTHER', 3, './train/OTHER/')
('YFT', 1, './train/YFT/')
('DOL', 5, './train/DOL/')
('LAG', 2, './train/LAG/')
('NoF', 0, './train/NoF/')
('SHARK', 4, './train/SHARK/')


In [ ]:
# General steps:
# Extract feature from each file as HOG or similar... or SIFT... or Similar...
# map each to feature space... and train some kind of classifier on that. SVM is a good choice.
# do the same for each feature in test set...
training_data = np.array([])
training_labels = np.array([])

for key in file_paths:
    cat = key[0]
    category = key[1]
    directory_path = key[2]
    file_list = file_paths[key]

    # shuffle this list, so we get random examples
    np.random.shuffle(file_list)

    # Stop early, while testing, so it doesn't take FOR-EV-ER (FOR-EV-ER)
    i = 0

    # read in the file and get its SIFT features
    for fname in file_list:
        fpath = directory_path + fname
        #print(fpath)
        #print("Category = " + str(category))
        # extract features!
        gray = cv2.imread(fpath,0)
        img=cv2.imread(fpath,0)
        gray = cv2.resize(gray, (400, 250))  # resize so we're always comparing same-sized images
                                             # Could also make images larger/smaller
                                             # to tune for greater accuracy / more speedd

        """ My Choice: SIFT (Scale Invariant Feature Transform)"""
        # However, this does not work on the Kaggle server
        # because it's in a separate package in the opencv version used on the Kaggle server.
        # This is a very robust method however, worth trying when it's reasonable to do so. 
        #detector = cv2.xfeatures2d.SIFT_create()
        #kp1, des1 = detector.detectAndCompute(gray, None)
        
        #Image saving
        #img=cv2.drawKeypoints(gray,kp1,img)
        #cv2.imwrite('./pictures/keypoints/'+cat+'-'+fname,img)
        
        """ Another option that will work on Kaggle server is ORB"""
        # find the keypoints with ORB
        #kp = cv2.orb.detect(img,None)
        # compute the descriptors with ORB
        #kp1, des1 = cv2.orb.compute(img, kp)

        """ Histogram of Gradients - often used to for detected people/animals in photos"""
        # Havent' tried this one in the SVM yet, but here's how to get the HoG, using openCV
        hog = cv2.HOGDescriptor()
        des1 = hog.compute(gray)
        # This is to make sure we have at least 100 keypoints to analyze
        # could also duplicate a few features if needed to hit a higher value
        #if len(kp1) < 100:
        #    continue

        # transform the data to float and shuffle all keypoints
        # so we get a random sampling from each image
        #des1 = des1.astype(np.float64)
        #np.random.shuffle(des1)
        #des1 = des1[0:100,:] # trim vector so all are same size
        vector_data = des1#.reshape(1,12800) 
        list_data = des1.tolist()

        # We need to concatenate ont the full list of features extracted from each image
        if len(training_data) == 0:
            training_data = np.append(training_data, vector_data)
            training_data = training_data.reshape(np.shape(vector_data))
        else:
            training_data   = np.concatenate((training_data, vector_data), axis=1)

        training_labels = np.append(training_labels,category)

        # early stop
        i += 1
        if i > 50:
            break
        


In [23]:
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
hog = cv2.HOGDescriptor()
#compute(img[, winStride[, padding[, locations]]]) -> descriptors
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
hist = hog.compute(gray)#,winStride,padding,locations)

print(np.shape(gray))
print(np.shape(hist))

(250, 400)
(2600640, 1)


In [3]:
print('Train data shape:', np.shape(training_data))
print('Train labels shape:', np.shape(training_labels))

Train data shape: (3679, 12800)
Train labels shape: (3679,)


In [4]:
        # Alright! Now we've got features extracted and labels
        X = training_data
        y = training_labels
        y = y.reshape(y.shape[0],)

        # Create and fit the SVM
        # Fitting should take a few minutes
        clf = svm.SVC(kernel='linear', C = 1.0, probability=True)
        clf.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [5]:
    # Now, extract one of the images and predict it
    gray = cv2.imread('./test_stg1/img_00071.jpg', 0)  # Correct is LAG --> Class 3
    img=cv2.imread('./test_stg1/img_00071.jpg', 0)
    img=cv2.drawKeypoints(gray,kp1,img)

    kp1, des1 = detector.detectAndCompute(gray, None)
    
    des1 = des1[0:100, :]   # trim vector so all are same size
    vector_data = des1.reshape(1, 12800)

    print("Linear SVM Prediction:")
    print(clf.predict(vector_data))        # prints highest probability class, only
    cv2.imwrite('./pictures/keypoints-test/'+int_cat[clf.predict(vector_data)[0]]+'-img_00071.jpg',img)
    print(clf.predict_proba(vector_data))  # shows all probabilities for each class. 
                                           #    need this for the competition

Linear SVM Prediction:
[ 6.]
[[ 0.09772263  0.20247569  0.01739086  0.09344943  0.04264332  0.01031217
   0.46582509  0.07018081]]


In [6]:
    
    # early stoppage...
    # only do 10
    i = 0
    prediction_output_list=[]
    for f in test_names:
        file_name = "./test_stg1/" + f
        #print("---Evaluating File at: " + file_name)
        if i%10==0:
            print(i)
        gray = cv2.imread(file_name, 0)
        #print(gray)
        gray = cv2.resize(gray, (400, 250))  # resize so we're always comparing same-sized images
        kp1, des1 = detector.detectAndCompute(gray, None)

        # ensure we have at least 100 keypoints to analyze
        if len(kp1) < 100:
            # and duplicate some points if necessary
            current_len = len(kp1)
            vectors_needed = 100 - current_len
            repeated_vectors = des1[0:vectors_needed, :]
            # concatenate repeats onto des1
            while len(des1) < 100:
                des1 = np.concatenate((des1, repeated_vectors), axis=0)
            # duplicate data just so we can run the model.
            des1[current_len:100, :] = des1[0:vectors_needed, :]

        np.random.shuffle(des1)  # shuffle the vector so we get a representative sample
        des1 = des1[0:100, :]   # trim vector so all are same size
        vector_data = des1.reshape(1, 12800)
        #print("Linear SVM Prediction:")
        #print(clf.predict(vector_data))
        svm_prediction = clf.predict_proba(vector_data)
        #print(svm_prediction)
        
        # format list for csv output
        csv_output_list = []
        csv_output_list.append(f)
        for elem in svm_prediction:      
            for value in elem:
                csv_output_list.append(value)

        # append filename to make sure we have right format to write to csv
        #print("CSV Output List Formatted:")
        #print(csv_output_list)

        # and append this file to the output_list (of lists)
        prediction_output_list.append(csv_output_list)

        # Uncomment to stop early
        #if i > 10:
        #    break
        i += 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [7]:
    # Write to csv
    print(prediction_output_list[0:5])
    #"""  Uncomment to write to your CSV. Can't do this on Kaggle server directly.
    try:
        with open("sift_and_svm_submission.csv", "w") as f:
            writer = csv.writer(f)
            f.flush()
            headers = ['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
            #headers= "image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT"
            writer.writerow(headers)
            writer.writerows(prediction_output_list)
    finally:
        f.close()
    #"""

[['img_01211.jpg', 0.20959866561786863, 0.18148281761244581, 0.031380833370168018, 0.063047982405357139, 0.034837293008693197, 0.02588047656980667, 0.3932368303982034, 0.060535101017457217], ['img_01371.jpg', 0.074426005010022264, 0.1913930969504597, 0.0033881815251937315, 0.083907244074230378, 0.0261945741080373, 0.017889014777113164, 0.53250447434851922, 0.070297409206424369], ['img_01847.jpg', 0.15011363255324109, 0.21491124095708508, 0.010937533552748034, 0.068730041346514725, 0.050715697502354946, 0.013332333360684642, 0.42692874448310364, 0.064330776244268029], ['img_01548.jpg', 0.12362203698598814, 0.24378210089724286, 0.0038552599029145749, 0.073551859880386428, 0.050049968921071032, 0.020605483045887538, 0.43973006439547496, 0.044803225971034137], ['img_00518.jpg', 0.20780249597354292, 0.15553881764499933, 0.004928935055606945, 0.082167945883271046, 0.045777425599947526, 0.01663673801049571, 0.44019634403434443, 0.046951297797792182]]


In [8]:
print(len(prediction_output_list))

1000


In [9]:
help(cv2.drawKeypoints)

Help on built-in function drawKeypoints:

drawKeypoints(...)
    drawKeypoints(image, keypoints, outImage[, color[, flags]]) -> outImage

